In [3]:
import pandas as pd
import numpy as np
import random

In [8]:
groups = ['Queen', 'Scorpions', 'Led Zeppelin', 'Black Sabbath', 'Deep Purple',
          'Kendrick Lamar', 'Kanye West', 'Tyler, the Creator', 'Jay-Z', '2pac']

In [23]:
group_df = pd.DataFrame(index=groups, columns=groups)
popularities = []
for i in range(len(groups) // 5):
    popularities.extend(range(5, 0, -1))


for i in range(len(groups) // 5):
    for j in range(5 * i):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 1
    for j in range(5 * i, 5 * i + 5):
        for k in range(5 * i, 5 * i + 5):
            if j != k:
                group_df.iloc[j, k] = group_df.iloc[k, j] = 3
            else:
                group_df.iloc[j, k] = 0
    for j in range(5 * i + 5, len(groups)):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 3

for i in range(len(groups)):
    group_df.iloc[i] *= popularities[i]
group_df

,Queen,Scorpions,Led Zeppelin,Black Sabbath,Deep Purple,Kendrick Lamar,Kanye West,"Tyler, the Creator",Jay-Z,2pac
Queen,0,15,15,15,15,5,5,5,5,5
Scorpions,12,0,12,12,12,4,4,4,4,4
Led Zeppelin,9,9,0,9,9,3,3,3,3,3
Black Sabbath,6,6,6,0,6,2,2,2,2,2
Deep Purple,3,3,3,3,0,1,1,1,1,1
Kendrick Lamar,5,5,5,5,5,0,15,15,15,15
Kanye West,4,4,4,4,4,12,0,12,12,12
"Tyler, the Creator",3,3,3,3,3,9,9,0,9,9
Jay-Z,2,2,2,2,2,6,6,6,0,6
2pac,1,1,1,1,1,3,3,3,3,0


In [24]:
transaction_count = 50_000
transactions = {'Transaction': [], 'Item': []}
for tr_id in range(transaction_count):
    items = random.choices(groups, weights=popularities)
    item_count = random.randint(1, 4)
    new_items = set(random.choices(groups, weights=group_df[items[0]], k=item_count - 1)).difference(set(items))
    items.extend(list(new_items))
    for item in items:
        transactions['Transaction'].append(tr_id + 1)
        transactions['Item'].append(item)
transactions = pd.DataFrame(transactions)
transactions

,Transaction,Item
0,1,Queen
1,1,Kendrick Lamar
2,2,Black Sabbath
3,3,Led Zeppelin
4,3,Kendrick Lamar
...,...,...
116296,49999,2pac
116297,49999,Kendrick Lamar
116298,50000,Kanye West
116299,50000,"Tyler, the Creator"


In [25]:
transactions.to_csv('groups-long.csv', index=False, encoding='utf-8')

# Анализ датасета

In [26]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [27]:
df = pd.read_csv("./groups-long.csv", encoding='utf-8')

In [28]:
df.head(100)

,Transaction,Item
0,1,Queen
1,1,Kendrick Lamar
2,2,Black Sabbath
3,3,Led Zeppelin
4,3,Kendrick Lamar
...,...,...
95,40,"Tyler, the Creator"
96,41,Scorpions
97,41,Black Sabbath
98,41,Deep Purple


In [29]:
df['Item'] = df['Item'].apply(lambda x: x.lower().strip())
df['Item'] = df.loc[df['Item'] != 'none', ['Item']]
df = df.dropna()

In [30]:
len(df['Item'].unique())

10

In [31]:
df['Transaction'].max() == len(df['Transaction'].unique())

np.True_

In [32]:
unique_items = df['Item'].unique()
_dict = defaultdict(list)
for transaction_number in tqdm(df['Transaction'].unique()):
    products = df.loc[df['Transaction'] == transaction_number]['Item'].to_list()
    for pr in unique_items:
        if pr in products:
            _dict[pr].append(1)
        else:
            _dict[pr].append(0)
norm_df = pd.DataFrame(_dict)
norm_df

100%|██████████| 50000/50000 [00:12<00:00, 3969.87it/s]


,queen,kendrick lamar,black sabbath,led zeppelin,"tyler, the creator",jay-z,scorpions,deep purple,kanye west,2pac
0,1,1,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,1,0,0,1,0,0,0
49996,0,0,0,0,0,0,1,0,0,0
49997,0,0,0,1,0,0,0,0,0,0
49998,0,1,0,1,0,0,0,0,0,1


In [33]:
norm_df.sum()

queen                 17931
kendrick lamar        17842
black sabbath          8450
led zeppelin          11946
tyler, the creator    12173
jay-z                  8523
scorpions             15301
deep purple            4639
kanye west            15102
2pac                   4394
dtype: int64

# Алгоритм Apriori

In [34]:
from itertools import combinations, permutations
from tqdm import tqdm

In [35]:
def count_rows(items: list[str]):
    condition = (norm_df[items[0]] == 1)
    for i in range(1, len(items)):
        condition &= (norm_df[items[i]] == 1)
    cnt = len(norm_df[condition])
    return cnt

In [36]:
k = 1
f = 4
results: list[list[set]] = []
while True:
    f_k: list[set] = []
    if k == 1:
        for item in unique_items:
            if norm_df.sum()[item] >= f:
                f_k.append({item})
    else:
        if k == 2:
            combs: list[set] = [x[0].union(x[1]) for x in combinations(results[0], r=2)]
        else:
            previous = results[k - 2]
            combs = []
            for i, elem in tqdm(enumerate(previous)):
                _set = set(sorted(elem)[:-1])
                if any(_set.issubset(x) for x in combs):
                    continue
                for j, elem1 in enumerate(previous):
                    if i == j:
                        continue
                    if _set.issubset(elem1) and all(any(len(set(subset).difference(x)) == 0 for x in previous) for subset in combinations(elem.union(elem1), r=k-1)):
                        combs.append(elem.union(elem1))
        
        for elem in combs:
            cnt = count_rows(list(elem))
            if cnt >= f:
                f_k.append(elem)
    
    if len(f_k) == 0:
        break
    results.append(f_k)
    k += 1

45it [00:00, 134146.18it/s]
8it [00:00, 38479.85it/s]


In [13]:
results

[[{'queen'},
  {'black sabbath'},
  {'deep purple'},
  {'led zeppelin'},
  {'scorpions'}],
 [{'black sabbath', 'queen'},
  {'deep purple', 'queen'},
  {'led zeppelin', 'queen'},
  {'black sabbath', 'led zeppelin'}],
 [{'black sabbath', 'led zeppelin', 'queen'}]]

In [37]:
results.pop(0)
while type(results[0]) != set:
    for elem in results[0]:
        results.append(elem)
    results.pop(0)
results

[{'kendrick lamar', 'queen'},
 {'black sabbath', 'queen'},
 {'led zeppelin', 'queen'},
 {'queen', 'tyler, the creator'},
 {'jay-z', 'queen'},
 {'queen', 'scorpions'},
 {'deep purple', 'queen'},
 {'kanye west', 'queen'},
 {'2pac', 'queen'},
 {'black sabbath', 'kendrick lamar'},
 {'kendrick lamar', 'led zeppelin'},
 {'kendrick lamar', 'tyler, the creator'},
 {'jay-z', 'kendrick lamar'},
 {'kendrick lamar', 'scorpions'},
 {'deep purple', 'kendrick lamar'},
 {'kanye west', 'kendrick lamar'},
 {'2pac', 'kendrick lamar'},
 {'black sabbath', 'led zeppelin'},
 {'black sabbath', 'tyler, the creator'},
 {'black sabbath', 'jay-z'},
 {'black sabbath', 'scorpions'},
 {'black sabbath', 'deep purple'},
 {'black sabbath', 'kanye west'},
 {'2pac', 'black sabbath'},
 {'led zeppelin', 'tyler, the creator'},
 {'jay-z', 'led zeppelin'},
 {'led zeppelin', 'scorpions'},
 {'deep purple', 'led zeppelin'},
 {'kanye west', 'led zeppelin'},
 {'2pac', 'led zeppelin'},
 {'jay-z', 'tyler, the creator'},
 {'scorpions

In [38]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in results:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[kendrick lamar],[queen],8.358,23.422262,83.755412,0.653122,-0.044390
1,[queen],[kendrick lamar],8.358,23.306006,83.860544,0.653122,-0.044390
2,[black sabbath],[queen],7.298,43.183432,112.886086,1.204156,0.012373
3,[queen],[black sabbath],7.298,20.350231,104.331753,1.204156,0.012373
4,[led zeppelin],[queen],10.376,43.428763,113.375636,1.210997,0.018079
...,...,...,...,...,...,...,...
109,"[kendrick lamar, queen]",[kanye west],2.150,25.723857,93.968262,0.851671,-0.003745
110,"[kanye west, queen]",[kendrick lamar],2.150,31.899110,94.442231,0.893933,-0.002551
111,"[kendrick lamar, 2pac]",[queen],0.528,14.095035,74.661575,0.393035,-0.008154
112,"[kendrick lamar, queen]",[2pac],0.528,6.317301,97.362694,0.718855,-0.002065


In [39]:
rules['Поддержка'].max()

np.float64(13.514000000000001)

In [40]:
rules['Достоверность'].max()

np.float64(44.491908321695554)

In [42]:
rules[(rules['Достоверность'] >= 30) & (rules['Лифт'] > 1)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
22,[kendrick lamar],"[tyler, the creator]",10.832,30.355341,108.628575,1.246831,0.021444
89,[2pac],[kanye west],2.974,33.841602,105.498323,1.120434,0.003197
75,[jay-z],[kanye west],6.006,35.234073,107.766541,1.166537,0.008574
79,[deep purple],[scorpions],3.286,35.417116,107.455715,1.157346,0.004467
40,[black sabbath],[scorpions],6.054,35.822485,108.134446,1.170593,0.008823
67,"[tyler, the creator]",[kanye west],8.734,35.874476,108.842776,1.187739,0.013805
52,[led zeppelin],[scorpions],8.732,36.547798,109.370516,1.194294,0.014206
30,[kendrick lamar],[kanye west],13.276,37.204349,111.147825,1.231769,0.024980
11,[queen],[scorpions],13.514,37.683342,111.363481,1.231401,0.025395
12,[deep purple],[queen],3.906,42.099590,110.772964,1.173933,0.005787


# Алгоритм Eclat

In [43]:
eclat_dict = {'Item': [], 'Transactions': []}
min_support = 4
for item in sorted(unique_items):
    transactions = norm_df[norm_df[item] == 1].index.to_list()
    eclat_dict['Item'].append([item])
    eclat_dict['Transactions'].append(transactions)
eclat_df = pd.DataFrame(eclat_dict)
eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]
eclat_df

,Item,Transactions
0,[2pac],"[19, 46, 48, 52, 53, 58, 83, 84, 90, 93, 109, ..."
1,[black sabbath],"[1, 3, 6, 7, 8, 12, 13, 23, 28, 30, 31, 34, 38..."
2,[deep purple],"[9, 19, 22, 24, 26, 27, 32, 40, 50, 55, 73, 78..."
3,[jay-z],"[7, 27, 28, 54, 56, 59, 60, 64, 76, 77, 80, 84..."
4,[kanye west],"[11, 18, 20, 26, 35, 39, 46, 47, 52, 56, 58, 6..."
5,[kendrick lamar],"[0, 2, 4, 5, 6, 11, 15, 19, 20, 21, 29, 33, 34..."
6,[led zeppelin],"[2, 11, 13, 14, 15, 16, 17, 21, 24, 27, 29, 31..."
7,[queen],"[0, 3, 7, 9, 11, 15, 16, 17, 19, 21, 23, 25, 3..."
8,[scorpions],"[8, 10, 16, 21, 22, 24, 25, 29, 30, 31, 32, 36..."
9,"[tyler, the creator]","[5, 18, 24, 25, 29, 39, 45, 54, 61, 62, 63, 64..."


In [44]:
min_support = 4
start_point = 0
k = 2
while True:
    end_point = len(eclat_df)
    
    new_dict = {'Item': [], 'Transactions': []}
    for i in tqdm(range(start_point, end_point)):
        for j in range(i + 1, end_point):
            item1, item2 = eclat_df.iloc[i]['Item'], eclat_df.iloc[j]['Item']
            if k > 2 and not all(x == y for x in item1[:k - 2] for y in item2[:k - 2]):
                break

            tr1, tr2 = eclat_df.iloc[i]['Transactions'], eclat_df.iloc[j]['Transactions']
            if len(set(item1).union(set(item2))) == k:
                new_dict['Item'].append(sorted(set(item1).union(set(item2))))
                new_dict['Transactions'].append(list(set(tr1).intersection(set(tr2))))
    
    eclat_df = pd.concat([
        eclat_df,
        pd.DataFrame(new_dict)
    ], ignore_index=True)
    if len(eclat_df) == end_point:
        break
    k += 1
    start_point = end_point
    eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]

eclat_df

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 120/120 [00:00<00:00, 30710.63it/s]


,Item,Transactions
0,[2pac],"[19, 46, 48, 52, 53, 58, 83, 84, 90, 93, 109, ..."
1,[black sabbath],"[1, 3, 6, 7, 8, 12, 13, 23, 28, 30, 31, 34, 38..."
2,[deep purple],"[9, 19, 22, 24, 26, 27, 32, 40, 50, 55, 73, 78..."
3,[jay-z],"[7, 27, 28, 54, 56, 59, 60, 64, 76, 77, 80, 84..."
4,[kanye west],"[11, 18, 20, 26, 35, 39, 46, 47, 52, 56, 58, 6..."
...,...,...
170,"[kendrick lamar, scorpions, tyler, the creator]","[38917, 34824, 22537, 22545, 8214, 22550, 4917..."
171,"[led zeppelin, queen, scorpions]","[49156, 32773, 16, 21, 24601, 16413, 32, 16417..."
172,"[led zeppelin, queen, tyler, the creator]","[49153, 28675, 49156, 49161, 26637, 2067, 1845..."
173,"[led zeppelin, scorpions, tyler, the creator]","[49156, 20496, 10258, 45075, 8214, 24, 29, 207..."


In [45]:
eclat_df = eclat_df[eclat_df['Item'].apply(lambda x: len(x) > 1)]

In [46]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in eclat_df['Item']:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[2pac],[black sabbath],0.800,9.103323,91.422484,0.538658,-0.006852
1,[black sabbath],[2pac],0.800,4.733728,95.744273,0.538658,-0.006852
2,[2pac],[deep purple],0.432,4.915794,95.412271,0.529833,-0.003834
3,[deep purple],[2pac],0.432,4.656176,95.666396,0.529833,-0.003834
4,[2pac],[jay-z],1.682,19.139736,102.589326,1.122829,0.001840
...,...,...,...,...,...,...,...
445,"[led zeppelin, tyler, the creator]",[scorpions],0.842,24.970344,92.494093,0.815971,-0.001899
446,"[scorpions, tyler, the creator]",[led zeppelin],0.842,18.921348,93.869346,0.791953,-0.002212
447,"[queen, scorpions]","[tyler, the creator]",1.320,9.767648,83.843542,0.401201,-0.019701
448,"[queen, tyler, the creator]",[scorpions],1.320,24.645258,92.095066,0.805348,-0.003190


In [47]:
rules['Поддержка'].max()

np.float64(13.514000000000001)

In [48]:
rules['Достоверность'].max()

np.float64(46.59949622166247)

In [49]:
rules[(rules['Достоверность'] >= 30) & (rules['Лифт'] > 1)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
76,[kendrick lamar],"[tyler, the creator]",10.832,30.355341,108.628575,1.246831,0.021444
399,"[kanye west, kendrick lamar]","[tyler, the creator]",4.070,30.656824,109.100859,1.259214,0.008378
166,"[2pac, tyler, the creator]",[kanye west],0.818,33.225020,104.524180,1.100021,0.000744
154,"[2pac, kendrick lamar]",[kanye west],1.264,33.742659,105.340780,1.117159,0.001326
6,[2pac],[kanye west],2.974,33.841602,105.498323,1.120434,0.003197
330,"[deep purple, led zeppelin]",[scorpions],0.876,34.569850,106.064253,1.129660,0.001005
267,"[black sabbath, led zeppelin]",[scorpions],1.580,34.801762,106.441527,1.137238,0.001907
213,"[black sabbath, deep purple]",[scorpions],0.610,34.817352,106.466984,1.137748,0.000739
358,"[jay-z, tyler, the creator]",[kanye west],1.634,35.034305,107.435163,1.159923,0.002253
48,[jay-z],[kanye west],6.006,35.234073,107.766541,1.166537,0.008574


# Алгоритм FP-Growth

In [50]:
item_count = {artist: count_rows([artist]) for artist in unique_items}
item_count

{'queen': 17931,
 'kendrick lamar': 17842,
 'black sabbath': 8450,
 'led zeppelin': 11946,
 'tyler, the creator': 12173,
 'jay-z': 8523,
 'scorpions': 15301,
 'deep purple': 4639,
 'kanye west': 15102,
 '2pac': 4394}

In [52]:
transaction_list: list[list[str]] = []
for trid in tqdm(df['Transaction'].unique()):
    transaction_list.append(df[df['Transaction'] == trid]['Item'].sort_values().to_list())
    transaction_list[-1].sort(key=lambda x: item_count[x], reverse=True)

100%|██████████| 50000/50000 [00:18<00:00, 2719.84it/s]


In [53]:
class FPTree:
    num: int = 0

    def __init__(self, value: str | None = None):
        self.value: str | None = value
        self.cnt: int = 0
        self.children: list['FPTree'] = []
        self.num: int = FPTree.num
        FPTree.num += 1
    
    def add(self, value: str):
        if value not in [x.value for x in self.children]:
            node = FPTree(value)
            node.cnt += 1
            self.children.append(node)
            return node
        else:
            index = [x.value for x in self.children].index(value)
            self.children[index].cnt += 1
            return self.children[index]
    
    def __str__(self):
        res = f'{self.num}({self.value}: {self.cnt}) -> [' + '  '.join([f'{x.num}({x.value}: {x.cnt})' for x in self.children]) + ']\n'
        for child in self.children:
            res += str(child)
        return res
    
    def find(self, value: str):
        result = dict.fromkeys(unique_items, 0)

        def dfs(node: 'FPTree'):
            cnt = 0
            for child in node.children:
                if child.value == value:
                    cnt = child.cnt
                    node.children.remove(child)
                else:
                    cnt += dfs(child)
                if node.value is not None:
                    result[node.value] += cnt
            return cnt
            
        while dfs(self) != 0:
            pass
        
        return result

In [54]:
root = FPTree()
node: FPTree = root
for transaction in transaction_list:
    for item in transaction:
        node = node.add(item)
    node = root
print(root)

0(None: 0) -> [1(queen: 17931)  3(black sabbath: 974)  4(kendrick lamar: 13663)  11(scorpions: 6160)  16(led zeppelin: 2033)  22(kanye west: 4976)  37(jay-z: 1085)  65(tyler, the creator: 2310)  93(2pac: 421)  103(deep purple: 447)]
1(queen: 17931) -> [2(kendrick lamar: 4179)  6(black sabbath: 784)  9(jay-z: 312)  13(deep purple: 323)  19(scorpions: 5649)  21(led zeppelin: 1941)  46(kanye west: 1589)  62(tyler, the creator: 913)  111(2pac: 83)]
2(kendrick lamar: 4179) -> [14(kanye west: 936)  18(led zeppelin: 438)  24(deep purple: 92)  27(scorpions: 1108)  45(black sabbath: 225)  54(2pac: 90)  137(jay-z: 240)  159(tyler, the creator: 550)]
14(kanye west: 936) -> [15(led zeppelin: 76)  59(2pac: 52)  92(tyler, the creator: 187)  125(black sabbath: 61)  197(deep purple: 33)  199(jay-z: 121)]
15(led zeppelin: 76) -> []
59(2pac: 52) -> []
92(tyler, the creator: 187) -> []
125(black sabbath: 61) -> []
197(deep purple: 33) -> []
199(jay-z: 121) -> []
18(led zeppelin: 438) -> [134(2pac: 16)  2

In [55]:
result = []
for item in sorted(unique_items, key=lambda x: item_count[x]):
    search_result = root.find(item)
    passed_threshold = [k for k, v in search_result.items() if v >= f]
    for i in range(1, len(passed_threshold) + 1):
        for comb in combinations(passed_threshold, r=i):
            result.append([item] + list(comb))
result

[['2pac', 'queen'],
 ['2pac', 'kendrick lamar'],
 ['2pac', 'black sabbath'],
 ['2pac', 'led zeppelin'],
 ['2pac', 'tyler, the creator'],
 ['2pac', 'jay-z'],
 ['2pac', 'scorpions'],
 ['2pac', 'deep purple'],
 ['2pac', 'kanye west'],
 ['2pac', 'queen', 'kendrick lamar'],
 ['2pac', 'queen', 'black sabbath'],
 ['2pac', 'queen', 'led zeppelin'],
 ['2pac', 'queen', 'tyler, the creator'],
 ['2pac', 'queen', 'jay-z'],
 ['2pac', 'queen', 'scorpions'],
 ['2pac', 'queen', 'deep purple'],
 ['2pac', 'queen', 'kanye west'],
 ['2pac', 'kendrick lamar', 'black sabbath'],
 ['2pac', 'kendrick lamar', 'led zeppelin'],
 ['2pac', 'kendrick lamar', 'tyler, the creator'],
 ['2pac', 'kendrick lamar', 'jay-z'],
 ['2pac', 'kendrick lamar', 'scorpions'],
 ['2pac', 'kendrick lamar', 'deep purple'],
 ['2pac', 'kendrick lamar', 'kanye west'],
 ['2pac', 'black sabbath', 'led zeppelin'],
 ['2pac', 'black sabbath', 'tyler, the creator'],
 ['2pac', 'black sabbath', 'jay-z'],
 ['2pac', 'black sabbath', 'scorpions'],
 ['

In [56]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in result:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        if count_rows(condition) == 0:
            continue
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[2pac],[queen],1.768,20.118343,80.291274,0.560993,-0.013836
1,[queen],[2pac],1.768,4.930009,95.941947,0.560993,-0.013836
2,[2pac],[kendrick lamar],3.746,42.626309,112.100160,1.194550,0.006101
3,[kendrick lamar],[2pac],3.746,10.497702,101.910233,1.194550,0.006101
4,[2pac],[black sabbath],0.800,9.103323,91.422484,0.538658,-0.006852
...,...,...,...,...,...,...,...
2521,"[scorpions, queen]",[kendrick lamar],2.216,16.397810,76.930999,0.459528,-0.026063
2522,"[scorpions, kendrick lamar]",[queen],2.216,31.729668,93.947104,0.884771,-0.002886
2523,"[queen, kendrick lamar]",[scorpions],2.216,26.513520,94.436419,0.866398,-0.003417
2524,[kendrick lamar],[queen],8.358,23.422262,83.755412,0.653122,-0.044390


In [57]:
rules.max()

Условие          [tyler, the creator, scorpions, kanye west]
Следствие                               [tyler, the creator]
Поддержка                                             13.514
Достоверность                                      46.599496
Убеждённость                                      120.440811
Лифт                                                1.344809
Рычаг                                               0.025395
dtype: object

In [58]:
rules['Достоверность'].max()

np.float64(46.59949622166247)

In [60]:
rules[(rules['Лифт'] >= 1) & (rules['Достоверность'] >= 30)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
2454,[kendrick lamar],"[tyler, the creator]",10.832,30.355341,108.628575,1.246831,0.021444
2473,"[kendrick lamar, kanye west]","[tyler, the creator]",4.070,30.656824,109.100859,1.259214,0.008378
105,"[2pac, tyler, the creator]",[kanye west],0.818,33.225020,104.524180,1.100021,0.000744
60,"[2pac, kendrick lamar]",[kanye west],1.264,33.742659,105.340780,1.117159,0.001326
16,[2pac],[kanye west],2.974,33.841602,105.498323,1.120434,0.003197
1149,"[deep purple, led zeppelin]",[scorpions],0.876,34.569850,106.064253,1.129660,0.001005
1795,"[black sabbath, led zeppelin]",[scorpions],1.580,34.801762,106.441527,1.137238,0.001907
1137,"[deep purple, black sabbath]",[scorpions],0.610,34.817352,106.466984,1.137748,0.000739
2185,"[jay-z, tyler, the creator]",[kanye west],1.634,35.034305,107.435163,1.159923,0.002253
2144,[jay-z],[kanye west],6.006,35.234073,107.766541,1.166537,0.008574
